This is just an example notebook for a reproducible demo for src/coincident/search/usgs_vendor_load.py as I test new functionality on the als_dem_fetch branch

In [1]:
import coincident
import geopandas as gpd
from shapely.geometry import box
from coincident.search.usgs_vendor_load import load_usgs_dem

%load_ext autoreload
%autoreload 2

/home/jehayes/coincident/src/coincident/search/main.py:14: UserWarning: Unable to authenticate with Maxar API. Please set MAXAR_API_KEY environment variable.
  from coincident.search import neon_api, opentopo_api, stac, wesm


In [2]:
# bounds from NEON site LIRO that has coincidence with USGS WI_12County_1_B22
# northern Wisconsin
aoi_test_bounds = [-89.72300752, 45.9840129, -89.68144678, 46.02525724]
aoi = gpd.GeoDataFrame(geometry=[box(*aoi_test_bounds)], crs="EPSG:4326")

In [3]:
gf = coincident.search.search(
    dataset="3dep",
    intersects=aoi,
    datetime=["2021", "2023"],
)

In [4]:
gf

,minx,maxx,miny,maxy,geometry,workunit,workunit_id,project,project_id,start_datetime,...,onemeter_reason,seamless_category,seamless_reason,lpc_link,sourcedem_link,metadata_link,collection,datetime,dayofyear,duration
fid,,,,,,,,,,,,,,,,,,,,,
2885,-90.046097,-88.931244,45.855568,46.300938,"POLYGON ((-88.93124 45.85557, -88.93124 46.300...",WI_12County_1_B22,230107,WI_12County_B22,230110,2022-05-04,...,Meets 3DEP 1-m DEM requirements,Meets,Meets 3DEP seamless DEM requirements,https://rockyweb.usgs.gov/vdelivery/Datasets/S...,http://prd-tnm.s3.amazonaws.com/index.html?pre...,http://prd-tnm.s3.amazonaws.com/index.html?pre...,3DEP,2022-05-09 12:00:00,129,11


In [5]:
load_usgs_dem?

Signature:
load_usgs_dem(
    aoi: geopandas.geodataframe.GeoDataFrame,
    project: str,
    tnmdataset: str = 'Digital Elevation Model (DEM) 1 meter',
    res: int = 1,
    clip: bool = True,
) -> xarray.core.dataarray.DataArray
Docstring:
Load and merge USGS 1-meter DEM tiles based on an AOI by querying the TNM API.

Steps:
  1. Reproject the AOI to EPSG:4326 (required by the TNM API).
  2. Convert the AOI into a polygon string for the API query.
  3. Query the TNM API for matching DEM products.
  4. Filter the API items by the given project identifier.
  5. Open each GeoTIFF as a lazy xarray.DataArray using rioxarray.
  6. Optionally coarsen the DEM data to a lower resolution by aggregating over 
     res x res blocks (default res=1 means no coarsening).
  7. Merge the DEM tiles by taking the maximum value along the new 'band' dimension 
     (preferring non-NaN values).
  8. Optionally clip the merged DEM mosaic to the original AOI geometry.

Parameters:
  aoi (gpd.GeoDataFrame): 

In [6]:
%%time
test_clip = load_usgs_dem(aoi, gf.iloc[0]["project"])

Polygon string for TNM API: -89.68144678 45.9840129, -89.68144678 46.02525724, -89.72300752 46.02525724, -89.72300752 45.9840129, -89.68144678 45.9840129
Querying TNM API with offset=0...
Found 2 products for project 'WI_12County_B22'.
CPU times: user 3.14 s, sys: 1.94 s, total: 5.08 s
Wall time: 9.06 s


In [7]:
test_clip

<xarray.DataArray 'elevation' (y: 4669, x: 4945)> Size: 92MB
dask.array<getitem, shape=(4669, 4945), dtype=float32, chunksize=(4235, 4945), chunktype=numpy.ndarray>
Coordinates:
  * x            (x) float64 40kB 2.9e+05 2.9e+05 ... 2.925e+05 2.925e+05
  * y            (y) float64 37kB 5.096e+06 5.096e+06 ... 5.1e+06 5.1e+06
    spatial_ref  int64 8B 0

In [8]:
%%time
test_no_clip = load_usgs_dem(aoi, gf.iloc[0]["project"], clip=False)

Polygon string for TNM API: -89.68144678 45.9840129, -89.68144678 46.02525724, -89.72300752 46.02525724, -89.72300752 45.9840129, -89.68144678 45.9840129
Querying TNM API with offset=0...
Found 2 products for project 'WI_12County_B22'.
CPU times: user 97.4 ms, sys: 6.45 ms, total: 104 ms
Wall time: 4.14 s


In [9]:
test_no_clip

<xarray.DataArray 'elevation' (y: 20024, x: 20024)> Size: 2GB
dask.array<_nanmax_skip-aggregate, shape=(20024, 20024), dtype=float32, chunksize=(5006, 5006), chunktype=numpy.ndarray>
Coordinates:
  * x            (x) float64 160kB 2.9e+05 2.9e+05 2.9e+05 ... 3e+05 3e+05 3e+05
  * y            (y) float64 160kB 5.09e+06 5.09e+06 ... 5.11e+06 5.11e+06
    spatial_ref  int64 8B 0

In [10]:
%%time
test_no_clip_coarse = load_usgs_dem(aoi, gf.iloc[0]["project"], clip=False, res=10)

Polygon string for TNM API: -89.68144678 45.9840129, -89.68144678 46.02525724, -89.72300752 46.02525724, -89.72300752 45.9840129, -89.68144678 45.9840129
Querying TNM API with offset=0...
Found 2 products for project 'WI_12County_B22'.
CPU times: user 87.5 ms, sys: 13.4 ms, total: 101 ms
Wall time: 4.15 s


In [11]:
test_no_clip_coarse

<xarray.DataArray 'elevation' (y: 2002, x: 2002)> Size: 16MB
dask.array<_nanmax_skip-aggregate, shape=(2002, 2002), dtype=float32, chunksize=(502, 502), chunktype=numpy.ndarray>
Coordinates:
  * x            (x) float64 16kB 2.9e+05 2.9e+05 2.9e+05 ... 3e+05 3e+05 3e+05
  * y            (y) float64 16kB 5.09e+06 5.09e+06 ... 5.11e+06 5.11e+06
    spatial_ref  int64 8B 0